# Домашнее задание 2. Классификация изображений.

In [21]:
import random
from tqdm import tqdm

import numpy as np

import torch
import torchvision
import torch.optim as optim
from torchvision import transforms
from torch import nn
from torch.nn import functional as F

from sklearn.metrics import accuracy_score

from PIL import Image
import albumentations as A

import os

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
#torch.manual_seed(SEED)
#torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# You may add any imports you need

### Результаты поиска пполезных материалов
https://pytorch.org/tutorials/beginner/basics/data_tutorial.html  <br>
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html <br>
https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html <br>

### Подготовка данных

In [22]:
class MyDataset(torch.utils.data.Dataset):
        
    def __init__(self, data_dir, transform):
        # YOUR CODE
        self.train_root = data_dir
        self.transforms = transform
        self.classes = []
        self.images = []      
               
        for img_class in os.listdir(self.train_root):
            img = os.listdir((os.path.join(self.train_root, img_class)))
            for i in img:
                self.images.append(self.train_root +'/'+ img_class + '/' + i)
                self.classes.append(img_class)      
        
    
    def __getitem__(self, idx):
        # YOUR CODE
        img = Image.open(self.images[idx]).convert("RGB")
        item = self.transforms(img)
        return item, int(self.classes[idx][6:])
        
        
    def __len__(self):
        # YOUR CODE
        return len(self.images)

In [12]:
transform = transforms.Compose(
    [
        #transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        #transforms.ColorJitter(hue=.05, saturation=.05), # делает обучение более стабильным
        transforms.RandomHorizontalFlip(),
        #transforms.RandomEqualize(),
        #transforms.RandomAffine(degrees=(30, 70), translate=(0.1, 0.3), scale=(0.5, 0.75)),
        #transforms.RandomRotation(degrees=(-90, 90)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

transform2 = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

train_transforms = transform
val_transforms = transform2

# YOU CAN DEFINE AUGMENTATIONS HERE



train_dataset = MyDataset(r"dataset/train", transform=train_transforms)
val_dataset = MyDataset(r"dataset/val", transform=val_transforms)

batch = 64
train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch, num_workers=4) # TRAIN DATALOADER WHICH YOU CONSTRUCT
val_dataloader = torch.utils.data.DataLoader(val_dataset, shuffle=True, batch_size=batch, num_workers=4) # VAL DATALOADER WHICH YOU CONSTRUCT

In [23]:
# Just very simple sanity checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[1][1], int)
print("tests passed")

tests passed


### Вспомогательные функции, реализация модели

In [24]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, epoch_index, device="cuda:0"):
    model.train()
    
    # YOUR CODE
    # TRAIN YOUR MODEL HERE
    running_loss = 0
    num = 0
    res_loss = []
    for x_, y_ in tqdm(train_dataloader):
        x_train, y_train = x_.to(device), y_.to(device) # перенос вычислений на GPU
        
        y_pred = model(x_train) # делаем предсказание
        
        loss = criterion(y_pred, y_train) 
        loss.backward()
        optimizer.step() # делаем шаг SGD
        optimizer.zero_grad() # зануляем градиенты
        
        running_loss += loss.item()
        res_loss.append(loss.item())
        num += 1
        if num % 250 == 249:    # 
            #print(f'[{epoch_index}, {num + 1:5d}] loss: {running_loss / 250:.3f}')
            running_loss = 0.0
    return np.mean(res_loss)

def predict(model, val_dataloader, criterion, device="cuda:0"):
    model.to(device)
    model.eval()
    # YOUR CODE
    # PREDICT FOR EVERY ELEMENT OF THE VAL DATALOADER AND RETURN CORRESPONDING LISTS
    losses = []
    true_classes = []
    predicted_classes = []
    total = 0
    correct = 0
    with torch.no_grad():
        for x_, y_ in tqdm(val_dataloader):
            x_test, y_test = x_.to(device), y_.to(device) # перенос вычислений на GPU           
            y_pred = model(x_test)
            loss = criterion(y_pred, y_test)
            losses.append(loss.item())
            
            _, predicted = torch.max(y_pred.data, 1)
            predicted_classes.extend(predicted.cpu().numpy().tolist())
            true_classes.extend(y_test.cpu().numpy().tolist())
            
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()
       
    return np.mean(losses), predicted_classes, true_classes


def train(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler=None, device="cuda:0", n_epochs=10):
    model.to(device)
    _, predicted, true = predict(model, val_dataloader, criterion, device="cuda:0")
    best_acc = accuracy_score(predicted, true)
    print(f"Начальная точность: {100 * best_acc:.2f} %" )
    for epoch in (range(1, n_epochs+1)):
        # YOUR CODE
        train_loss = train_one_epoch(model, train_dataloader, criterion, optimizer, epoch, device="cuda:0")
        if True or epoch % 2 == 1: 
            valid_loss, predicted, true = predict(model, val_dataloader, criterion, device="cuda:0")
            accuracy = accuracy_score(predicted, true)
            print(f"Эпоха: {epoch};\t train loss: {train_loss:.3f};\t valid loss: {valid_loss:.3f};\t Точность: {100 * accuracy:.2f} %")
            if best_acc < accuracy:
                best_acc = accuracy
                torch.save(model, "model/MyModel_baseline_best.pth")
                torch.save(model.state_dict(), "model/My_model_baseline_best_w.pt")
                
        if scheduler != None:
            scheduler.step()

### Обучение модели, запуски экспериментов

In [25]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = nn.Sequential()
              
        self.linear_layers = nn.Sequential()

         # Антецедент распространения
    def forward(self, x):
        x = self.cnn_layers(x)
        #x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

    
model = torch.load("model/MyModel_baseline_best.pth")
model.eval()

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(6, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(6, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15):

In [26]:
n_epochs = 30
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.005, eps=1e-08) # YOUR OPTIMIZER

criterion = nn.CrossEntropyLoss() # LOSS THAT YOU OPTIMIZE (SHOLD BE CROSS ENTROPY OR SMTH ELSE)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.97)

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

Простой тест на проверку правильности написанного кода

In [27]:
torch.cuda.empty_cache()
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(predicted_labels, true_labels)
print("tests passed")
print(accuracy)

100%|██████████| 157/157 [00:10<00:00, 15.65it/s]

tests passed
0.33


Запустить обучение можно в ячейке ниже.

In [28]:
train(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler, device, n_epochs)

100%|██████████| 157/157 [00:09<00:00, 16.81it/s]


Начальная точность: 33.00 %


100%|██████████| 157/157 [00:07<00:00, 19.63it/s]


Эпоха: 1;	 train loss: 2.506;	 valid loss: 2.898;	 Точность: 32.64 %


100%|██████████| 157/157 [00:07<00:00, 20.49it/s]


...

Эпоха: 18;	 train loss: 2.486;	 valid loss: 2.900;	 Точность: 32.83 %


100%|██████████| 157/157 [00:07<00:00, 20.29it/s]


Эпоха: 19;	 train loss: 2.486;	 valid loss: 2.890;	 Точность: 33.04 %


 49%|████▉     | 772/1563 [00:55<00:57, 13.84it/s]


KeyboardInterrupt: 

In [29]:
torch.save(model, "model/my_model2.pth")

In [30]:
#all_losses, predictions, labels = predict(model, val_dataloader, criterion, device)

classes = [x for x in range(200)]

# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for images_, labels_ in val_dataloader:
        images, labels = images_.to(device), labels_.to(device)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname} is {accuracy:.1f} %')

Accuracy for class: 0 is 60.0 %
Accuracy for class: 1 is 24.0 %
Accuracy for class: 2 is 72.0 %
Accuracy for class: 3 is 2.0 %
Accuracy for class: 4 is 26.0 %
Accuracy for class: 5 is 34.0 %
Accuracy for class: 6 is 12.0 %
Accuracy for class: 7 is 28.0 %
Accuracy for class: 8 is 22.0 %
Accuracy for class: 9 is 36.0 %
Accuracy for class: 10 is 18.0 %
Accuracy for class: 11 is 10.0 %
Accuracy for class: 12 is 32.0 %
Accuracy for class: 13 is 36.0 %
Accuracy for class: 14 is 22.0 %
Accuracy for class: 15 is 30.0 %
Accuracy for class: 16 is 40.0 %
Accuracy for class: 17 is 28.0 %
Accuracy for class: 18 is 62.0 %
Accuracy for class: 19 is 62.0 %
Accuracy for class: 20 is 46.0 %
Accuracy for class: 21 is 20.0 %
Accuracy for class: 22 is 34.0 %
Accuracy for class: 23 is 34.0 %
Accuracy for class: 24 is 42.0 %
Accuracy for class: 25 is 8.0 %
Accuracy for class: 26 is 58.0 %
Accuracy for class: 27 is 24.0 %
Accuracy for class: 28 is 74.0 %
Accuracy for class: 29 is 8.0 %
Accuracy for class: 30 